<a href="https://colab.research.google.com/github/HanSeungJun/Spam_filter/blob/main/%EC%A4%91%EA%B0%84%EA%B3%A0%EC%82%AC2_%ED%95%9C%EC%8A%B9%EC%A4%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**인공지능 / 중간고사2 / 201802873 / 한승준**

# Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20.

In [4]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ensembles"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 연습문제 7-8

MNIST 데이터를 불러들여 훈련세트, 검증 세트, 테스트 세트로 나누세요. (Ex. 훈련에 50,000개 샘플, 검증에 10,000개 샘플, 테스트에 10,000개 샘플). 그럼 다음 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기 같은 여러 종류의 분류기를 훈련, 그리고 검증 세트에서 개개의 분류기보다 더 높은 성능을 내도록 이들을 간접 또는 직접 "투표" 방법을 사용해 앙상블로 연결해보기.
앙상블로 얻고 나면 테스트 세트로 확인. 개개의 분류기와 비교해서 성능이 얼마나 향상되나? 

In [5]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)

The MNIST data set 를 불러옵니다.

In [6]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(
    mnist.data, mnist.target, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=10000, random_state=42)

MNIST 데이터셋을 Train과 Test용으로 각각 만개씩 분류합니다.

Random Forest 분류기, Extra-Trees 분류기, 선형 SVM 및 MLP 분류기와 같은 다양한 분류기들로 이제 훈련해보자.

In [7]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [8]:
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(max_iter=100, tol=20, random_state=42)
mlp_clf = MLPClassifier(random_state=42)

In [9]:
# 한 번에 여러 분류기들을 훈련 시켜보자
models = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]

for model in models:
    print("훈련 :", model)
    model.fit(X_train, y_train)

훈련 : RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
훈련 : ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None,

In [12]:
# 4가지 분류기들의 score 값을 각각 보자
[model.score(X_val, y_val) for model in models]

[0.9692, 0.9715, 0.8662, 0.9639]

위와 같은 결과를 얻었습니다.


하드 또는 소프트 투표 분류기를 사용하여 검증 세트에서 어떤 결과가 나오는지 앙상블로 결합해 보겠습니다.

In [13]:
# 투표 분류기 
from sklearn.ensemble import VotingClassifier

named_models = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    ("svm_clf", svm_clf),
    ("mlp_clf", mlp_clf),
]

In [14]:
voting_clf = VotingClassifier(named_models)

In [15]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_lea

In [16]:
# 앙상블 학습의 결과 
voting_clf.score(X_val, y_val)

0.9702

In [17]:
# 앙상블 학습에 들어간 개별적인 4개의 모델들에 대한 결과
[model.score(X_val, y_val) for model in models]

[0.9692, 0.9715, 0.8662, 0.9639]

위와 같은 결과가 나왔는데 성능이 아주 미세하게 떨어졌습니다.

정확도를 높이기 위하여 가장 낮은 정확도를 가진 SVM을 투표 분류기에서 제거하여 성능 향상 여부를 보겠습니다. 

set_params ()를 사용하여 None으로 설정시 SVM을 제거 할 수 있습니다. 

`None` using `set_params()` like this:

In [18]:
#SVM 제거
voting_clf.set_params(svm_clf=None)

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_lea

In [19]:
voting_clf.estimators_

[RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=100,
                        n_jobs=None, oob_score=False, random_state=42, verbose=0,
                        warm_start=False),
 ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                      criterion='gini', max_depth=None, max_features='auto',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs

In [20]:
# 안지워져서 확실하게 다시 지워주겠습니다.
del voting_clf.estimators_[2]

`VotingClassifier`의 score를 확인하겠습니다.

In [21]:
voting_clf.score(X_val, y_val)

0.9736

SVM를 제거한 결과 성능이 올라갔습니다. 0.9702 -> 0.9736

이것은 SVM이 성능 저하를 시켰다고 볼 수 있습니다.

이제 소프트 투표 분류기로 결과를 확인해보겠습니다.

이것은 간접투표에 해당하며 훈련은 이미 위에서 진행했습니다.

In [22]:
# 간접투표
voting_clf.voting = "soft"

In [23]:
# 간접투표 결과
voting_clf.score(X_val, y_val)

0.97

하드 투표 방식은 0.9736 의 점수를 얻은 반면,

소포트 투표 방식은 0.97로 더 낮은 점수를 보여주고 있습니다.

자 그러면 이제 MNIST Test data를 이용하여 지금까지
만들어진 분류기들을 총 평가 및 비교 하겠습니다.

In [24]:
# 앙상블 학습 hard
voting_clf.voting = "hard"
voting_clf.score(X_test, y_test)

0.9704

In [25]:
# 앙상블 학습 soft
voting_clf.voting = "soft"
voting_clf.score(X_test, y_test)

0.9671

In [26]:
# 사용한 모델들 ( RandomForestClassifier, ExtraTreesClassifier, LinearSVC, MLPClassifier)

[model.score(X_test, y_test) for model in models]

[0.9645, 0.9691, 0.866, 0.9604]

[Test data 결과] 

투표 분류기를 사용할 경우, 각각의 분류기들의 장점과 단점들을 서로 보안할 수 있어 정확도가 더 상승할 수 있었습니다.